## Mobility data

This notebook processes mobility data. Currently we're focusing on reading Google's mobility report, and only doing so at a national level.

#### Note

This uses the output of the `generate_countries_geojson` notebook.

### Papermill

In [ ]:
# parameters
data_dir = '/opt/src/data'

For papermill execution, the pameters are:

- data_dir: That data directory to read data from and publish data to.

In [ ]:
import io
import os
import json
from collections import defaultdict

import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import mapping
from shapely.algorithms.polylabel import polylabel
import pycountry
from jenkspy import jenks_breaks

### Compute the mobility data

In [ ]:
GOOGLE_MOBILITY_URL = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'

google_df = pd.read_csv(GOOGLE_MOBILITY_URL)

In [ ]:
google_df

#### Notes about the data

country_region_code is the ISO 3166-1 alpha-2 for the country. sub_region_1 is the region; if this is null, it represents the country totals. sub_region_2 is only used for US counties.

For starters we're only processing country totals.

In [ ]:
google_country_df = google_df[google_df['sub_region_1'].isnull()]

Construct the dataset, renaming the property names to be a little more terse.

In [ ]:
value_cols = {
    'retail_and_recreation_percent_change_from_baseline': 'retail_and_recreation', 
    'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_and_pharmacy',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline': 'transit_stations', 
    'workplaces_percent_change_from_baseline': 'workplaces', 
    'residential_percent_change_from_baseline': 'residential'
}

In [ ]:
countries_gdf = gpd.read_file(os.path.join(data_dir, 'published/countries.geojson'))

In [ ]:
countries_gdf[countries_gdf['NAME'] == 'France']

Mapbox needs IDs to be integers for features in order to use setFeatureState properly. We'll match the country ISO2 code to the country ID in the GeoJSON and use it as the key into the country data.

In [ ]:
alpha3_to_id = {}
for _, row in countries_gdf.iterrows():
    alpha3_to_id[row['ADM0_A3']] = row['id']


Gather mobility data and print if there's any issues finding countries.

Réunion is not in the country geojson and is part of France; skipping for now.

In [ ]:
mobility_data = {}
country_codes = set([])
issue_found = set([])
dates = set([])
for _, row in google_country_df.iterrows():
    alpha_3 = None
    # I think pandas is reading the alpha_2 'NA' incorrectly...
    if row['country_region'] == 'Namibia':        
        alpha_2 = row['country_region_code']
        alpha_3 = 'NAM'
    else:
        alpha_2 = row['country_region_code']
        country = pycountry.countries.get(alpha_2=alpha_2)
        if country is None:
            print('CANNOT FIND {}'.format(row['country_region']))
        alpha_3 = country.alpha_3
      
    if alpha_3 is not None and alpha_3 in alpha3_to_id:
        feature_id = alpha3_to_id[alpha_3]
        country_codes.add(alpha_3)
        date = row['date']
        dates.add(date)
        if not feature_id in mobility_data:
            mobility_data[feature_id] = {}
        mobility_data[feature_id]['code'] = alpha_3
        for col, prop in value_cols.items():
            if not prop in mobility_data[feature_id]:
                mobility_data[feature_id][prop] = {}
            v = row[col]
            if np.isnan(v):
                v = None
            mobility_data[feature_id][prop][date] = v
    else:
        if not row['country_region'] in issue_found:
            print('Problem with {} ({})'.format(row['country_region'], alpha_3))
            issue_found.add(row['country_region'])


Special consideration for EAC: do a per-capita average of the EAC regions to develop regional scores, using the alpha3 code 'EAC'

In [ ]:
eac_id = max(alpha3_to_id.values()) + 1
alpha3_to_id['EAC'] = eac_id

eac_country_codes = [
    'BDI', # Burundi
    'SSD', # South Sudan
    'RWA', # Rawanda
    'TZA', # Tanzania
    'UGA', # Uganda
    'KEN' # Kenya
]

eac_populations = {
    'BDI': 11745876, # Source: 2019 Estimate https://en.wikipedia.org/wiki/Burundi
    'SSD': 10975927, # Source: 2019 Estimate https://en.wikipedia.org/wiki/South_Sudan    
    'RWA': 12374397, # Source: 2019 Estimate https://en.wikipedia.org/wiki/Rwanda 
    'TZA': 56313438, # Source: 2018 Estimate https://en.wikipedia.org/wiki/Tanzania 
    'UGA': 42729036, # Source: 2019 Estimate https://en.wikipedia.org/wiki/Uganda 
    'KEN': 47564296 # Source: 2019 Census https://en.wikipedia.org/wiki/Kenya
}

eac_mobility_data_list = defaultdict(list)
for md in mobility_data.values():
    if md['code'] in eac_country_codes:
        for prop in value_cols.values():            
            eac_mobility_data_list[prop].append((md[prop], md['code']))

eac_mobility_data = {}
for prop, values in eac_mobility_data_list.items():
    eac_mobility_data[prop] = {}
    for date in dates:
        v = 0
        i = 0.0
        for (c_values, code) in values:   
            if date in c_values:
                c_v = c_values[date]
                pop = eac_populations[code]
                v += (c_v * pop)
                i += pop
        if i > 1:
            eac_mobility_data[prop][date] = int(v / i)

eac_mobility_data

In [ ]:
mobility_data[eac_id] = eac_mobility_data
mobility_data[eac_id]['code'] = 'EAC'
alpha3_to_id['EAC'] = eac_id

Write out the mobility data.

In [ ]:
with open(os.path.join(data_dir, 'published/google_mobility_data.json'), 'w') as f:
    f.write(json.dumps(mobility_data, sort_keys=True))

In [ ]:
with open(os.path.join(data_dir, 'published/country_alpha_3_to_id.json'), 'w') as f:
    f.write(json.dumps(alpha3_to_id, sort_keys=True))

In [ ]:
mobility_data[7]

### Compute the breaks

In [ ]:
def compute_breaks(mb_data, per_capita_base=None):
    result = {}
    # Tracking positive and negative values to compute separate breakpoints
    prop_values = defaultdict(lambda: {'neg': [0], 'pos': [0] })
    for region in mb_data:
        for prop in value_cols.values():
            for date in mb_data[region][prop]:
                v = mb_data[region][prop][date]
                if v is not None and not np.isnan(v):
                    if v < 0:
                        prop_values[prop]['neg'].append(v)
                    else:
                        prop_values[prop]['pos'].append(v)
            
    for prop in prop_values:
        neg_breaks = jenks_breaks(prop_values[prop]['neg'], nb_class=4)
        pos_breaks = jenks_breaks(prop_values[prop]['pos'], nb_class=4)
        result[prop] = neg_breaks + pos_breaks[1:]

    return result

In [ ]:
country_breaks = compute_breaks(mobility_data)
country_breaks

### Generate configuration

This configuration is used in the Mobility tab.

Currently light on config, but set up to handle multiple aggregation levels and other configuration.

In [ ]:
config = {
    'dates': sorted(dates),
    'aggregations': {
        'country': {
            'breaks': country_breaks
        }
    }    
}

In [ ]:
with open(os.path.join(data_dir, 'published/mobility-config.json'), 'w') as f:
    f.write(json.dumps(config, indent=4, sort_keys=True))